In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft, fftfreq

data = np.load("first_thread.npy")

VELA_PERIOD = 89  # ms

In [ ]:
51_200_000 / 27

In [ ]:
OVERLAP = 0
SEGMENT = 1024
SAMPLING_RATE = 1_900_000  # Hz


window = np.hanning(SEGMENT)

n_segments = (data.shape[1] - SEGMENT) // (SEGMENT - OVERLAP)

frequencies_axis = fftfreq(SEGMENT, 1 / SAMPLING_RATE)
frequencies_axis = np.fft.fftshift(frequencies_axis)
times = np.arange(n_segments) * (SEGMENT - OVERLAP) / SAMPLING_RATE

In [ ]:
n_segments

In [ ]:
output = np.empty((8, data.shape[1] // SEGMENT, SEGMENT))

for channel in range(8):
    for j in range(n_segments):
        start = j * (SEGMENT - OVERLAP)
        end = start + SEGMENT

        sliced_data = data[channel, start:end]
        if len(sliced_data) < SEGMENT:
            print(len(sliced_data))
            continue
        ff_transform = np.abs(fft(sliced_data * window))

        output[channel, start:end, :] = ff_transform

In [ ]:
ff_transform

In [ ]:
len(times)

In [ ]:
output[0].shape

In [ ]:
n_channels = 8


fig, axs = plt.subplots(n_channels, 1, figsize=(20, 16), sharex=True, sharey=True)

for i in range(n_channels):
    ax = axs[i]
    im = ax.imshow(
        output[i],
        aspect="auto",
        cmap="inferno",
        origin="lower",
        extent=[times[0], times[-1], frequencies_axis[0], frequencies_axis[-1]],
    )
    ax.set_ylabel(f"Channel {i + 1} Frequency [Hz]")
    ax.set_xlabel("Time [s]")


fig.colorbar(im, ax=axs, label="Amplitude")


plt.suptitle("Frequency vs Time for Multiple Channels", fontsize=16)


# plt.tight_layout()
plt.show()

In [ ]:
output[0].T.shape

In [ ]:
# len(times) is number of data points in SEGMENT time
# so this gives the number of SEGMENT (FFT) required to get a period of
# 89ms
round(len(times) * 89 / 1000 / (times[-1] - times[0]))

In [ ]:
times

In [ ]:
# Check on above number - approximately correct.
51200000 / 27 * 89 / 1000 / (SEGMENT - OVERLAP)

In [ ]:
len(times) / 2642

In [ ]:
output.shape

In [ ]:
FOLD_SEGMENT = 2642

output_folded = np.zeros((n_channels, FOLD_SEGMENT, 128))
for channel in range(n_channels):
    for i in range(0, len(times), FOLD_SEGMENT):
        start = i * FOLD_SEGMENT
        end = start + FOLD_SEGMENT

        if end > len(times):
            break
        output_folded[channel, :] += output[channel, start:end]

In [ ]:
fig, axs = plt.subplots(n_channels, 1, figsize=(20, 16), sharex=True, sharey=True)

for i in range(n_channels):
    ax = axs[i]
    im = ax.imshow(
        output_folded[i].T,
        aspect="auto",
        cmap="inferno",
        origin="lower",
        extent=[times[0], times[-1], frequencies_axis[0], frequencies_axis[-1]],
    )
    ax.set_ylabel(f"Channel {i + 1} Frequency [Hz]")
    ax.set_xlabel("Time [s]")


fig.colorbar(im, ax=axs, label="Amplitude")


plt.suptitle("Frequency vs Time for Multiple Channels", fontsize=16)


# plt.tight_layout()
plt.show()

In [ ]:
output_folded.shape

In [ ]:
## use scipy
from scipy.signal import ShortTimeFFT
from scipy.signal.windows import gaussian

g_std = 512
w = gaussian(1024, std=g_std, sym=True)  # symmetric Gaussian window
SFT = ShortTimeFFT(w, hop=512, fs=SAMPLING_RATE, fft_mode="centered")
Sx = SFT.stft(data[7])  # perform the STFT

In [ ]:
N = len(data[0])

T_x = 1 / SAMPLING_RATE
t_x = np.arange(N) * T_x


fig1, ax1 = plt.subplots(figsize=(6.0, 4.0))  # enlarge plot a bit
t_lo, t_hi = SFT.extent(N)[:2]  # time range of plot
ax1.set_title(
    rf"STFT ({SFT.m_num * SFT.T:g}$\,s$ Gaussian window, "
    + rf"$\sigma_t={g_std * SFT.T}\,$s)"
)
ax1.set(
    xlabel=f"Time $t$ in seconds ({SFT.p_num(N)} slices, "
    + rf"$\Delta t = {SFT.delta_t:g}\,$s)",
    ylabel=f"Freq. $f$ in Hz ({SFT.f_pts} bins, "
    + rf"$\Delta f = {SFT.delta_f:g}\,$Hz)",
    xlim=(t_lo, t_hi),
)

im1 = ax1.imshow(
    abs(Sx), origin="lower", aspect="auto", extent=SFT.extent(N), cmap="viridis"
)
# ax1.plot(t_x, f_i, 'r--', alpha=.5, label='$f_i(t)$')
fig1.colorbar(im1, label="Magnitude $|S_x(t, f)|$")

# Shade areas where window slices stick out to the side:
for t0_, t1_ in [
    (t_lo, SFT.lower_border_end[0] * SFT.T),
    (SFT.upper_border_begin(N)[0] * SFT.T, t_hi),
]:
    ax1.axvspan(t0_, t1_, color="w", linewidth=0, alpha=0.2)
for t_ in [0, N * SFT.T]:  # mark signal borders with vertical line:
    ax1.axvline(t_, color="y", linestyle="--", alpha=0.5)
ax1.legend()
fig1.tight_layout()
plt.show()